## Capstone Project

### Part 1
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [64]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import requests

### 1. Import URL and check table

In [65]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {}

r = requests.get(url, headers=header)

tables = pd.read_html(r.text)

df=pd.DataFrame(tables[0])

df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 2. Clean up data 
- remove boroughs that are N/A
- combine neighborhoods if postal codes are the same
- if there is no neighborhood but a borough, neighborhood will be the same as borough

In [66]:
# Data frame has 3 columns: PostalCode, Borough, and Neighborhood
# No boroughs that are N/A

df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [67]:
# combine neighbourhoods if postal codes are the same
df["Neighborhood"] = df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

# emove duplicates
df = df.drop_duplicates()

# update index to be postcode if it isn't already
if(df.index.name != 'Postal Code'):
    df = df.set_index('Postal Code')
    
df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [68]:
# If there is no neighbourhood but a borough, neighbourhood will be the same as borough
df['Neighborhood'].replace("Not assigned", df["Borough"],inplace=True)
df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### 3. .Shape method to print the number of rows of dataframe

In [69]:
df.shape

(103, 2)

### Part 2
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [81]:
# Get coordinates from csv
coordinates_df=pd.read_csv("http://cocl.us/Geospatial_data")

# Rename columns
coordinates_df.columns = ["Postcode", "Latitude", "Longitude"]
if(coordinates_df.index.name != 'Postcode'):
    coordinates_df = coordinates_df.set_index('Postcode')
    
coordinates_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [89]:
df.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
